In [1]:
import networkx as nx
import pynauty as na

In [2]:
g = na.Graph(number_of_vertices=8, directed=False,
          adjacency_dict = { 0: [1,2],
                             1: [0,4],
                             2: [0,3,4,5],
                             3: [2,4,5,6],
                             4: [1,2,3,5],
                             5: [4,2,3,7],
                             6: [3,7],
                             7: [5,6] })

h = na.Graph(number_of_vertices=8, directed=False,
          adjacency_dict = { 0: [1,2],
                             1: [0,4],
                             2: [0,3,4,5],
                             3: [2,4,5,7],
                             4: [1,2,3,5],
                             5: [4,2,3,6],
                             6: [5,7],
                             7: [3,6] })

i = na.Graph(number_of_vertices=8, directed=False,
          adjacency_dict = { 0: [2,4,3,6],
                             1: [5,4],
                             2: [0,3,4,5],
                             3: [0,2,4,7],
                             4: [0,1,2,3],
                             5: [1,2],
                             6: [0,7],
                             7: [3,6] })

j = na.Graph(number_of_vertices=8, directed=False,
          adjacency_dict = { 0: [1,2],
                             1: [0,3],
                             2: [0,3,4,5],
                             3: [1,2,4,5],
                             4: [2,3,5,6],
                             5: [2,3,4,5,6,7],
                             6: [4,5,7],
                             7: [5,6] })

print("g = h? ", na.certificate(g) == na.certificate(h), "g = i? ", na.certificate(g) == na.certificate(i), "g = h? ", na.certificate(g) == na.certificate(j))

g = h?  True g = i?  True g = h?  False


In [5]:
num_graphs = 4
random_nx_graphs = [ nx.gnp_random_graph(8,0.4) for i in range(num_graphs) ]
random_na_graphs = [ na.Graph(number_of_vertices = 8, directed = False,
                              adjacency_dict = { n: list(nbrdict.keys()) for n, nbrdict in graph.adjacency() }
                              ) for graph in random_nx_graphs
                   ]

nx_iso = [ nx.is_isomorphic(random_nx_graphs[i],random_nx_graphs[j]) for i in range(num_graphs) for j in range(i,num_graphs) ]
na_iso = [ na.certificate(random_na_graphs[i]) == na.certificate(random_na_graphs[j]) for i in range(num_graphs) for j in range(i,num_graphs) ]

print("Are the isomorphisms the same? ", nx_iso == na_iso)
print("What do the isomorphism matrices look like?\n", nx_iso, na_iso)

Are the isomorphisms the same?  True
What do the isomorphism matrices look like?
 [True, False, False, False, True, False, False, True, False, True] [True, False, False, False, True, False, False, True, False, True]


In [17]:
num_graphs = 1000
graph_size = 4
random_nx_graphs = [ nx.gnp_random_graph(graph_size,0.6) for i in range(num_graphs) ]
random_na_graphs = [ na.Graph(number_of_vertices = graph_size, directed = False,
                              adjacency_dict = { n: list(nbrdict.keys()) for n, nbrdict in graph.adjacency() }
                              ) for graph in random_nx_graphs
                   ]

In [18]:
%%time
nx_iso = [ nx.is_isomorphic(random_nx_graphs[i],random_nx_graphs[j]) for i in range(num_graphs) for j in range(i,num_graphs) ]

CPU times: user 33.3 s, sys: 515 ms, total: 33.8 s
Wall time: 37.9 s


In [19]:
%%time
na_iso = [ na.certificate(random_na_graphs[i]) == na.certificate(random_na_graphs[j]) for i in range(num_graphs) for j in range(i,num_graphs) ]

CPU times: user 4.55 s, sys: 60.9 ms, total: 4.61 s
Wall time: 5.43 s


In [37]:
import networkx as nx
import pynauty as na
import networkx.algorithms.isomorphism as iso

def find_type_match(graph, graphlet_list):
    """
    Given a graph, find an isomorphism with one of the canonical graphs from
    'graphlet_list'.
    Return index of the corresponding graph from 'graphlet_list' and a
    match dictionary.
    The match dictionary has format {u_i: v_i}, 'u_i' are nodes from 'graph'
    and 'v_i' are nodes from canonical graph.
    Helper function for 'prob_functions' for unordered method.
    """
    nodes = graph.nodes()
    n = len(nodes)
    if n == 1:
        # trivial graph: just send it to zero!
        return (0, {u: 0 for u in nodes})
    if n == 2:
        # 2-path graph: both nodes are equal, pick a random isomorphism
        return (0, {u: i for i, u in enumerate(nodes)})
    if n == 3:
        if graph.number_of_edges() == 2:
            # wedge-graph: find root, other two are arbitrary
            u0 = next((node for node in nodes if graph.degree(node) == 2))
            (u1, u2) = (node for node in graph.neighbors(u0))
            return (0, {u0: 0, u1: 1, u2: 2})
        if graph.number_of_edges() == 3:
            # triangle: all three are arbitrary
            return (1, {u: i for i, u in enumerate(nodes)})
    if n == 4:
        e_num = graph.number_of_edges()
        max_degree = max((graph.degree(node) for node in nodes))
        if e_num == 3 and max_degree == 3:
            u3 = next((node for node in nodes if graph.degree(node) == 3))
            (u0, u1, u2) = tuple(graph.neighbors(u3))
            return (0, {u0: 0, u1: 1, u2: 2, u3: 3})
        if e_num == 3 and max_degree == 2:
            (u0, u1) = (node for node in nodes if graph.degree(node) == 2)
            u2 = next((node for node in graph.neighbors(u1) if node != u0))
            u3 = next((node for node in graph.neighbors(u0) if node != u1))
            return (1, {u0: 0, u1: 1, u2: 2, u3: 3})
        if e_num == 4 and max_degree == 3:
            u3 = next((node for node in nodes if graph.degree(node) == 3))
            (u1, u2) = (node for node in nodes if graph.degree(node) == 2)
            u0 = next((node for node in nodes if graph.degree(node) == 1))
            return (2, {u0: 0, u1: 1, u2: 2, u3: 3})
        if e_num == 4 and max_degree == 2:
            u0 = next((node for node in nodes))
            (u1, u3) = tuple(graph.neighbors(u0))
            u2 = next((node for node in graph.neighbors(u1) if node != u0))
            return (3, {u0: 0, u1: 1, u2: 2, u3: 3})
        if e_num == 5:
            (u0, u2) = (node for node in nodes if graph.degree(node) == 3)
            (u1, u3) = (node for node in nodes if graph.degree(node) == 2)
            return (4, {u0: 0, u1: 1, u2: 2, u3: 3})
        if e_num == 6:
            (u0, u1, u2, u3) = tuple(nodes)
            return (5, {u0: 0, u1: 1, u2: 2, u3: 3})
        raise ValueError("wrong graphlet format")

    # Improve matching procedure here for n>4.
    for (i, graph_) in enumerate(graphlet_list):
        graph_matcher = iso.GraphMatcher(graph, graph_)
        if graph_matcher.is_isomorphic():
            break
    #assert graph_id[1].is_isomorphic()
    return (i, graph_matcher.mapping)

def get_graphlet_list(k):
    """
    Generate list of all graphlets of size 'k'.
    List is taken from graph_atlas of networkx.
    """
    from networkx.generators.atlas import graph_atlas_g
    assert k > 0
    atlas = graph_atlas_g()[1:]
    graphlet_list = []
    for graph in atlas:
        n = graph.number_of_nodes()
        if n < k:
            continue
        if n > k:
            break
        if nx.is_connected(graph):
            graphlet_list.append(graph)
    return graphlet_list

def nxgraph_to_nagraph(nxgraph):
    return na.Graph(number_of_vertices = graphlet_size, 
                    directed = False,
                    adjacency_dict = { n: list(nbrdict.keys()) 
                                       for n, nbrdict in nxgraph.adjacency() }
                    )

graphlet_size = 3
num_graphs = 100
random_nx_graphs = [ nx.gnp_random_graph(graphlet_size, 0.4) for i in range(num_graphs) ]
random_na_graphs = [ nxgraph_to_nagraph(graph) for graph in random_nx_graphs ]

graphlet_list = get_graphlet_list(graphlet_size)
na_graphlet_list = [ nxgraph_to_nagraph(graph) for graph in random_nx_graphs ]

SyntaxError: invalid syntax (<ipython-input-39-d71aa03aac5d>, line 1)

In [34]:
%%timeit
nx_types = [ find_type_match(graph, graphlet_list) for graph in random_nx_graphs ]

4.36 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
